In [1]:
!pip install casadi
from IPython.display import clear_output
import os

import numpy as np
import casadi
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation

os.makedirs('images',exist_ok=True)
if os.name == 'nt':
    plt.rcParams['font.family'] = 'MS Gothic'
elif os.name == 'posix':
    !pip install japanize-matplotlib


clear_output()

In [2]:
g = 9.81 #重力加速度
M = 1 #台車の質量
m = 0.2 #振子の質量
l = 1 #振子の長さ

nu = 1 #制御変数の次元
nx = 4 #状態変数の次元

# コスト関数の重み
Q = casadi.diag([2.5,10,0.01,0.01])
Q_f = casadi.diag([2.5,10,0.01,0.01])
R = casadi.diag([0.1])

# 予測ホライズン等
T = 1
K = 20
dt = T/K

# 制約条件
x_lb = [-np.inf,-np.inf,-np.inf,-np.inf]
x_ub = [np.inf,np.inf,np.inf,np.inf]
u_lb = [-15]
u_ub = [15]

total = nx*(K+1) + nu*K

In [3]:
def make_f():
    states = casadi.SX.sym("states",nx)
    ctrls = casadi.SX.sym("ctrls",nu)

    x = states[0]
    theta = states[1]
    x_dot = states[2]
    theta_dot = states[3]
    F = ctrls[0]

    sin = casadi.sin(theta)
    cos = casadi.cos(theta)
    det = M+m*sin**2

    x_ddot = (-m*l*sin*theta_dot**2+m*g*sin*cos+F)/det
    theta_ddot = (-m*l*sin*cos*theta_dot**2+(M+m)*g*sin+F*cos)/(l*det)

    states_dot = casadi.vertcat(x_dot,theta_dot,x_ddot,theta_ddot)

    f = casadi.Function("f",[states,ctrls],[states_dot],['x','u'],['x_dot'])
    return f

def make_RK4():
    states = casadi.SX.sym("states",nx)
    ctrls = casadi.SX.sym("ctrls",nu)

    f = make_f()

    r1 = f(x=states,u=ctrls)["x_dot"]
    r2 = f(x=states+dt*r1/2,u=ctrls)["x_dot"]
    r3 = f(x=states+dt*r2/2,u=ctrls)["x_dot"]
    r4 = f(x=states+dt*r3,u=ctrls)["x_dot"]

    states_next = states + dt*(r1+2*r2+2*r3+r4)/6

    RK4 = casadi.Function("RK4",[states,ctrls],[states_next],["x","u"],["x_next"])
    return RK4

def make_integrator():
    states = casadi.SX.sym("states",nx)
    ctrls = casadi.SX.sym("ctrls",nu)

    f = make_f()
    ode = f(x=states, u=ctrls)["x_dot"]

    dae = {"x":states,"p":ctrls,"ode":ode}

    I = casadi.integrator("I","cvodes",dae,0,dt)
    return I

def compute_stage_cost(x,u):
    cost = (casadi.dot(Q@x,x) + casadi.dot(R@u,u)) / 2
    return cost

def compute_final_cost(x):
    cost = casadi.dot(Q_f@x,x) / 2
    return cost

In [4]:
d = 2

# tau = np.append(0, casadi.collocation_points(d, 'legendre'))

# C = np.zeros((d+1,d+1))
# beta = np.zeros(d+1)
# B = np.zeros(d+1)

# for j in range(d+1):
#     p = np.poly1d([1])
#     for r in range(d+1):
#         if r != j:
#             p *= np.poly1d([1, -tau[r]]) / (tau[j]-tau[r])

#     beta[j] = p(1.0)

#     pder = np.polyder(p)
#     for r in range(d+1):
#         C[j,r] = pder(tau[r])

#     pint = np.polyint(p)
#     B[j] = pint(1.0)
A = np.zeros((d,d))
b = np.zeros(d)
tau = casadi.collocation_points(d, 'legendre')
for j in range(d):
    lj = 1
    for l in range(d):
        if l != j:
            lj *= np.poly1d([1, -tau[l]]) / (tau[j]-tau[l])
    lj_int = np.polyint(lj)
    b[j] = lj_int(1)-lj_int(0)
    for i in range(d):
        A[i,j] = lj_int(tau[i])-lj_int(0)
D = np.linalg.inv(A)
beta = D.T@b

In [5]:
print("beta",beta)

print("A",A)
print("invA",np.linalg.inv(A))
print("b",b)
print("D",D)
print("D.T@b",D.T@b)


NameError: name 'B' is not defined

In [ ]:
def make_nlp_dc():
    f = make_f()

    U = [casadi.SX.sym(f"u_{i}",nu) for i in range(K)]
    X = [casadi.SX.sym(f"x_{i}",nx) for i in range(K+1)]
    Xc = []
    for k in range(K):
        for j in range(d):
            Xc.append( casadi.SX.sym(f"x_{k}_{j}",nx) )
        
    G = []
    J = 0

    for k in range(K):
        Xk_end = 1*X[k]
        for j in range(d):
            xp = 0
            for r in range(d):
                xp = xp + D[j,r]*(Xc[k*d+r]-X[k])

            fj = f(Xc[k*d+j],U[k])
            Xk_end = Xk_end + beta[j]*Xc[k*d+j];
            J = J + dt*b[j]*compute_stage_cost(Xc[k*d+j],U[k])
            G.append(dt*fj - xp)
        G.append(Xk_end-X[k+1])
    
    J += compute_final_cost(X[-1])
    option = {'print_time':False}
    nlp = {"x":casadi.vertcat(*X,*Xc,*U),"f":J,"g":casadi.vertcat(*G)}
    S = casadi.nlpsol("S","ipopt",nlp,option)
    return S

In [ ]:
def compute_optimal_control_dc(S_dc,x_init,x0_dc):
    x_init = x_init.full().ravel().tolist()
    
    lbx = x_init + x_lb*K*(1+d) + u_lb*K
    ubx = x_init + x_ub*K*(1+d) + u_ub*K
    lbg = [0]*nx*K*(1+d)
    ubg = [0]*nx*K*(1+d)

    res = S_dc(lbx=lbx,ubx=ubx,lbg=lbg,ubg=ubg,x0=x0_dc)
    
    offset = nx + nx*K*(1+d)
    x0_dc = res["x"]
    u_opt = x0_dc[offset:offset+nu]
    return u_opt, x0_dc

In [ ]:
S_dc = make_nlp_dc()

In [ ]:
def make_nlp():
    RK4 = make_RK4()

    U = [casadi.SX.sym(f"u_{i}",nu) for i in range(K)]
    X = [casadi.SX.sym(f"x_{i}",nx) for i in range(K+1)]
    G = []

    J = 0

    for k in range(K):
        J += compute_stage_cost(X[k],U[k]) * dt
        eq = X[k+1] - RK4(x=X[k],u=U[k])["x_next"]
        G.append(eq)
    J += compute_final_cost(X[-1])

    option = {'print_time':False}
    nlp = {"x":casadi.vertcat(*X,*U),"f":J,"g":casadi.vertcat(*G)}
    S = casadi.nlpsol("S","ipopt",nlp,option)
    return S

In [ ]:
def compute_optimal_control(S,x_init,x0):
    x_init = x_init.full().ravel().tolist()
    
    lbx = x_init + x_lb*K + u_lb*K
    ubx = x_init + x_ub*K + u_ub*K
    lbg = [0]*nx*K
    ubg = [0]*nx*K

    res = S(lbx=lbx,ubx=ubx,lbg=lbg,ubg=ubg,x0=x0)
    
    offset = nx*(K+1)
    x0 = res["x"]
    u_opt = x0[offset:offset+nu]
    return u_opt, x0

In [ ]:
S = make_nlp()

In [ ]:
x_init = casadi.DM([0,np.pi,0,0])

x0_dc = casadi.DM.zeros(nx + nx*K*(1+d) + nu*K)
u_opt, x0_dc = compute_optimal_control_dc(S_dc,x_init,x0_dc)

x0 = casadi.DM.zeros(total)
u_opt, x0 = compute_optimal_control(S,x_init,x0)

In [ ]:
timestamp = np.array(range(K))* dt
plt.plot(timestamp, x0[nx*(K+1):],label="multiple-shooting")
plt.plot(timestamp, x0_dc[nx + nx*K*(1+d):],label="collocation")
plt.xlabel("Time [s]")
plt.ylabel("Control Input [N]")
plt.legend()
plt.savefig("images/chap6_collocation_control_trajectory.png")